## Exercise 1 : JSON

In [6]:
import urllib.request
import json

In [3]:
url="http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.geojson"
response=urllib.request.urlopen(url)


In [4]:
json_str =response.read().decode('utf-8')

In [8]:
json_parsed_str=json.loads(json_str)
json_parsed_str.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

In [9]:
json_parsed_str['metadata']

{'generated': 1574136851000,
 'url': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.geojson',
 'title': 'USGS Significant Earthquakes, Past Month',
 'status': 200,
 'api': '1.8.1',
 'count': 13}

In [10]:
title=json_parsed_str['metadata']['title']
title

'USGS Significant Earthquakes, Past Month'

In [12]:
quakes=json_parsed_str['features']

In [14]:
quake1=quakes[0]
quake1.keys()

dict_keys(['type', 'properties', 'geometry', 'id'])

In [21]:
print(json.dumps(quake1,indent=10))

{
          "type": "Feature",
          "properties": {
                    "mag": 5.9,
                    "place": "2km N of Kadingilan, Philippines",
                    "time": 1574083329943,
                    "updated": 1574132947083,
                    "tz": 480,
                    "url": "https://earthquake.usgs.gov/earthquakes/eventpage/us700069f4",
                    "detail": "https://earthquake.usgs.gov/earthquakes/feed/v1.0/detail/us700069f4.geojson",
                    "felt": 163,
                    "cdi": 6.4,
                    "mmi": 7.07,
                    "alert": "green",
                    "status": "reviewed",
                    "tsunami": 0,
                    "sig": 640,
                    "net": "us",
                    "code": "700069f4",
                    "ids": ",us700069f4,",
                    "sources": ",us,",
                    "types": ",dyfi,geoserve,ground-failure,losspager,moment-tensor,origin,phase-data,shakemap,",
             

## Exercise 2 : Unicode

In [1]:
15

15

In [2]:
0xFF

255

In [3]:
0x7F

127

In [4]:
0xF

15

In [5]:
'\u0394'

'Δ'

In [6]:
"\U00000394"

'Δ'

In [7]:
"\N{GREEK CAPITAL LETTER DELTA}"

'Δ'

In [8]:
b'\xFFab'

b'\xffab'

In [9]:
type(b'ab')

bytes

In [10]:
type(b'\xFFab')

bytes

In [11]:
len(b'\xFFab')

3

In [12]:
u=chr(40960) + 'abcd' + chr(1972)

type(u)

u

print(u)

ꀀabcd޴


In [13]:
u.encode('ascii','ignore')

b'abcd'

In [14]:
u.encode('ascii','replace')

b'?abcd?'

In [15]:
u.encode('ascii', 'xmlcharrefreplace')

b'&#40960;abcd&#1972;'

In [16]:
u.encode('ascii', 'backslashreplace')

b'\\ua000abcd\\u07b4'

## Exercise 3 : Mongo DB

In [18]:
from pymongo import MongoClient

In [19]:
client = MongoClient('localhost', 27017)

In [21]:
client.list_database_names()

['admin', 'config', 'local']

In [22]:
db=client.peopledb

In [34]:
client.list_database_names()

['admin', 'config', 'local', 'peopledb']

In [25]:
db.list_collection_names()

[]

In [27]:
peoplecoll = db.people

In [28]:
type(peoplecoll)

pymongo.collection.Collection

In [30]:
peoplelist = [{ "name": "John Smith", "age": 30 }, { "name": "Bo\
Bennett", "age": 23 }, { "name": "Anna Jones", "age": 25 }]

In [32]:
peoplecoll.insert_one({ "name": "John Smith", "age": 30 })

In [33]:
db.people

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'peopledb'), 'people')

In [35]:
 peoplecoll.insert_many(peoplelist[1: ])

In [36]:
docs = peoplecoll.find()

In [37]:
type(docs)

pymongo.cursor.Cursor

In [38]:
for doc in docs:
    print(doc)

{'_id': ObjectId('5dd3d82d671414bbfb6923f0'), 'name': 'John Smith', 'age': 30}
{'_id': ObjectId('5dd3d840671414bbfb6923f1'), 'name': 'John Smith', 'age': 30}
{'_id': ObjectId('5dd3d888671414bbfb6923f2'), 'name': 'BoBennett', 'age': 23}
{'_id': ObjectId('5dd3d888671414bbfb6923f3'), 'name': 'Anna Jones', 'age': 25}


In [39]:
peoplelist2=[{ "name": "Britney Sykes", "age": 21 , 'position':'Guard'}, { "name": "Briana Day", "age": 19, 'position':'Center'}, { "name": "Alexis Peterson", "age": 21, 'position':'Guard' }, { "name": "Gabby Cooper", "age": 18, 'position':'Guard'}]

In [40]:
 peoplecoll.insert_many(peoplelist2)

In [42]:
docs = peoplecoll.find()
for doc in docs:
    if doc['age']<20:
        print(doc)

{'_id': ObjectId('5dd47719671414bbfb6923f5'), 'name': 'Briana Day', 'age': 19, 'position': 'Center'}
{'_id': ObjectId('5dd47719671414bbfb6923f7'), 'name': 'Gabby Cooper', 'age': 18, 'position': 'Guard'}


## Excercise 4: MongoDB and JSON

In [43]:
''' 
	This program accesses the USGS earthquake data feed. This URL is for
       significant quakes in the last 30 days.
	It then saves the earthquakes to a database named usgs and a collection called earthquakes.
	The mongod server must be running for this program to work!
'''

import urllib.request
import json
import pymongo


# get the bbc rss feed of news stories and connect to it
earthquake_url = "http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.geojson"

try:
	response = urllib.request.urlopen(earthquake_url)
except urllib.error.URLError as e:
    if hasattr(e, 'reason'):
        print('We failed to reach a server.')
        print('Reason: ', e.reason)
    elif hasattr(e, 'code'):
        print('The server couldn\'t fulfill the request.')
        print('Error code: ', e.code)
else:
	# the url request was successful - convert the response to a string
	json_string = response.read().decode('utf-8')

	# the json package loads() converts the string to python dictionaries and lists
	eq_json = json.loads(json_string)
	
	# from the json dictionary we get the title to print
	title = eq_json['metadata']['title']
	print('Collected data from', title)
	#  and we get the list of earthquakes
	quakelist = eq_json['features']

	# Connection to Mongo DB
	try:
	    client=pymongo.MongoClient('localhost', 27017)
	    print ("Connected successfully!!!")
	except pymongo.errors.ConnectionFailure as e:
	   print ("Could not connect to MongoDB: %s" % e )
	else:

		# use database named usgs or create it if not there already
		eqdb = client.usgs
		# create collection named earthquakes or create it if not there already
		quakecoll = eqdb.earthquakes
		# add all the earthquakes to the list
		quakecoll.insert_many(quakelist)
		print("Added", len(quakelist), "to earthquakes collection in usgs database")
		# close the database connection
		client.close()



Collected data from USGS Significant Earthquakes, Past Month
Connected successfully!!!
Added 12 to earthquakes collection in usgs database


In [49]:
client=pymongo.MongoClient('localhost', 27017)
eqdb = client.usgs
quakecoll = eqdb.earthquakes
docs=quakecoll.find()
for doc in docs:
    print('Place:{}, Time:{}'.format(doc['properties']['place'],doc['properties']['time']))

Place:2km N of Kadingilan, Philippines, Time:1574083329943
Place:138km E of Bitung, Indonesia, Time:1573748260457
Place:26km W of Pepeekeo, Hawaii, Time:1573490152790
Place:167km SSW of Ndoi Island, Fiji, Time:1573209884876
Place:57km NE of Hashtrud, Iran, Time:1573166825428
Place:6km W of Ventura, CA, Time:1573131939400
Place:East of the South Sandwich Islands, Time:1572987121296
Place:136km W of Neiafu, Tonga, Time:1572907412280
Place:30km SW of Illapel, Chile, Time:1572904405320
Place:0km NW of Bulatukan, Philippines, Time:1572484279120
Place:14km E of Bual, Philippines, Time:1572311084738
Place:North of Ascension Island, Time:1571846894497


In [50]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)